In [9]:
from stock_data import Stock
import statsmodels.tsa.stattools as stattools
import pandas as pd


In [16]:
stocks = [Stock("A").five_minute(),
          Stock("AAL").five_minute()]


_, p, _ =stattools.coint(stocks[0]["price"], stocks[1]["price"])

In [21]:
stattools.coint(stocks[0]["price"], stocks[1]["price"])

(-1.968100337322799,
 0.5451148353125662,
 array([-3.89867785, -3.33737801, -3.04531619]))

In [22]:
x,y,z = (1,2,3)

In [23]:
x

1